In [1]:
import matplotlib
matplotlib.use('TkAgg')  # Set Matplotlib backend to TkAgg (for interactive plotting on some systems)

import numpy as np
import matplotlib.pyplot as plt  # Standard plotting
from matplotlib.animation import FuncAnimation  # Optional: for animations

from shapely.geometry import LineString, Point  # For geometric computations
from shapely.strtree import STRtree  # For spatial indexing of geometries

import networkx as nx # reconstruct network

import random  # For Python’s own RNG (not used yet)

plt.rc("text", usetex=False)  # Use standard text rendering (no LaTeX)
plt.rc("font", family = "serif")  # Set font to serif
plt.rc("figure", figsize=(10,8))  # Default figure size

%config InlineBackend.figure_format = 'retina'  # Jupyter setting for high-res inline plots (won’t work in script mode)

In [2]:
class Particle:
    def __init__(self, x, y, p_run, p_tumble, theta, speed, p_link, L_link_mean, L_link_std):
        self.position = np.array([x, y])  # Current position (in unit square)
        self.p_run = p_run  # Probability of running at each timestep
        self.p_tumble = p_tumble  # Probability of tumbling (direction change)
        self.theta = theta  # Current heading (angle in radians)
        self.speed = speed  # Speed per timestep
        self.p_link = p_link  # Probability of laying a link at each timestep
        self.L_link_mean = L_link_mean  # Mean link length
        self.L_link_std = L_link_std  # Std dev of link length

    def maybe_lay_link(self):
        if np.random.rand() < self.p_link:
            # Sample link length from Gaussian, fold negative to positive
            L_link = np.abs(np.random.normal(self.L_link_mean, self.L_link_std))
            return True, L_link
        return False, 0.0

    def run(self, dt):
        if np.random.rand() < self.p_run:
            displacement = self.speed * dt # how far does the particle run?
            dx = displacement * np.cos(self.theta) # change in x coordinate (w/ orientation)
            dy = displacement * np.sin(self.theta) # change in y coordinate (w/ orientation)
            self.position = (self.position + np.array([dx, dy])) % 1.0  # Enforce periodic boundaries

    def tumble(self):
        if np.random.rand() < self.p_tumble:
            self.theta = np.random.uniform(-np.pi, np.pi)  # Randomly sample new direction

edges = []  # List of edges as ((x1, y1), (x2, y2)) tuples
nodes = []  # List of node positions (redundant in current logic)

def add_link(particle, L_link):
    # Compute the endpoints of the link centered at particle's position and aligned with heading
    dx = 0.5 * L_link * np.cos(particle.theta)
    dy = 0.5 * L_link * np.sin(particle.theta)
    start = (particle.position - np.array([dx, dy])) % 1.0  # Periodic wrap
    end = (particle.position + np.array([dx, dy])) % 1.0 # Periodic wrap
    edges.append((start, end))  # Store edge
    nodes.append(particle.position.copy())  # Currently adds a node at every deposition (may not be desired)

def simulate(N, steps, dt, particle_params):
    global nodes, edges
    nodes, edges = [], []  # Clear previous run data

    # Create N particles with randomized initial positions and angles
    particles = [
        Particle(
            x=np.random.rand(), y=np.random.rand(),  # Random position
            theta=np.random.uniform(-np.pi, np.pi),  # Random direction
            **particle_params  # Fill in other parameters
        )
        for _ in range(N)
    ]

    for step in range(steps):
        for p in particles:
            laid_link, L_link = p.maybe_lay_link()
            if laid_link:
                add_link(p, L_link)  # Add link if randomly chosen
            p.run(dt)  # Move forward (maybe)
            p.tumble()  # Change direction (maybe)

    # Placeholder for optional post-processing (like merging links)
    #edges[:] = merge_collinear_links(edges)

def plot_network(nodes, edges, step=None):
    plt.figure(figsize=(6, 6))
    for start, end in edges:
        if np.linalg.norm(np.array(start) - np.array(end)) > 0.5:
            continue  # Skip links that wrap around the periodic boundary
        x_vals = [start[0], end[0]]
        y_vals = [start[1], end[1]]
        plt.plot(x_vals, y_vals, 'b-', alpha=0.5)  # Draw edge as blue line

    if nodes:
        xs, ys = zip(*nodes)
        plt.scatter(xs, ys, color='red', s=10, label='Nodes')  # Draw nodes as red dots

    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.gca().set_aspect('equal')  # Keep square aspect ratio
    plt.title(f"Network at Step {step}" if step is not None else "Network")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

In [3]:
# 1. Define particle behavior parameters
params = {
    'p_run': 0.5,           # 50% chance of running
    'p_tumble': 0.5,        # 50% chance of tumbling
    'speed': 0.05,          # Distance per time step
    'p_link': 0.2,          # 20% chance of laying a link per step
    'L_link_mean': 0.1,     # Average link length
    'L_link_std': 0.02      # Std deviation of link length
}

# 2. Run the simulation
steps = 1000
N = 10
dt = 1.0
simulate(N=N, steps=steps, dt=dt, particle_params=params)
plot_network(nodes, edges, step=steps)